# Step1: NYT Corpus Processing

**Data Size: 1652347**

In [1]:
import re
import pyarrow.feather as feather
total_nyt_data = feather.read_feather("./0_Corpus/0_NYT_Data_Extraction/0_NYT_Data/0_NYT_total_data.feather")

print(f"raw nyt_pd Size: {len(total_nyt_data)}")
print(total_nyt_data.head(1).values)

FileNotFoundError: [WinError 3] Failed to open local file './0_Corpus/0_NYT_Data_Extraction/0_NYT_Data/0_NYT_total_data.feather'. Detail: [Windows error 3] 系统找不到指定的路径。


In [2]:
def filtering_NYT_data(raw_nyt_pd):
    #1.Remove None body_text Records
    before_filtering_step_data_num = len(raw_nyt_pd)
    raw_nyt_pd = raw_nyt_pd[raw_nyt_pd.body_text != "No_Text"]
    remove_doc_num = before_filtering_step_data_num-len(raw_nyt_pd)
    
    companyrep_paraid = []
    bad_short_paraid = []
    duplicate_doc_num = 0
    for index, row in raw_nyt_pd.iterrows():
        file_id = row['file_id']
        pub = row['pub']
        title = row['title']
        body_text = row['body_text'].strip()
        
        body_text=re.sub(r"^LEAD:","",body_text).strip()
        
        #2.Remove COMPANY REPORTS Records
        if "*** COMPANY REPORTS **" in body_text:
            companyrep_paraid.append(file_id)
            continue
        
        #3.Remove bad short paragraph Records
        token_num = body_text.count(" ")+1
        para_num = body_text.count("\n")+1
        if token_num/para_num<=15:
            bad_short_paraid.append(file_id)
            continue
        
        paragraph_list=body_text.split("\n")
        #4.Modified duplicate leading paragraph Records
        if len(paragraph_list)>1:
            lead_para=paragraph_list[0]
            second_para=paragraph_list[1]
            if second_para in lead_para or lead_para in second_para:
                body_text="\n".join(paragraph_list[1:])
                duplicate_doc_num+=1
        raw_nyt_pd.at[index,'body_text'] = body_text
    
    companyrep_paraid = set(companyrep_paraid)
    bad_short_paraid = set(bad_short_paraid)
    print(f"1.Removed {remove_doc_num} records that do not have body_text.")
    print(f"2.Remove {len(companyrep_paraid)} records that have COMPANY REPORTS.")
    print(f"3.Remove {len(bad_short_paraid)} records that are short paragraphs.")
    print(f"4.Modified {duplicate_doc_num} records that have duplicate leading paragraph.")
    
    remove_id_list = list(set.union(companyrep_paraid, bad_short_paraid))
    raw_nyt_pd = raw_nyt_pd[~raw_nyt_pd['file_id'].isin(remove_id_list)]
    
    before_filtering_data_num = len(raw_nyt_pd)
    raw_nyt_pd=raw_nyt_pd[~raw_nyt_pd.duplicated(['body_text'],keep='last')]
    remove_doc_num = before_filtering_data_num-len(raw_nyt_pd)
    print(f"5.removed {remove_doc_num} records that have duplicate body_text (keep only the last).")
    
    filtered_nyt_pd = raw_nyt_pd.reset_index(drop=True)
    return filtered_nyt_pd

In [3]:
filtered_nyt_pd = filtering_NYT_data(total_nyt_data)
print(len(filtered_nyt_pd))

1.Removed 24549 records that do not have body_text.
2.Remove 103251 records that have COMPANY REPORTS.
3.Remove 47582 records that are short paragraphs.
4.Modified 316145 records that have duplicate leading paragraph.
5.removed 27929 records that have duplicate body_text (keep only the last).
1652347


# Step2: Shuffle & Save Split Files

**"./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/{i}_NYT_filtered_data.feather"**

In [4]:
filtered_nyt_pd = filtered_nyt_pd.sample(frac=1).reset_index(drop=True)
split_num = len(filtered_nyt_pd)//10
for i in range(10):
    if i==9:
        sampled_nyt_pd = filtered_nyt_pd.iloc[i*split_num:].reset_index(drop=True)
    else:
        sampled_d_feather(nyt_file)
    s = nyt_data.title.str.len().sort_values().index
    sort_nyt_data = nyt_data.reindex(s)nyt_pd = filtered_nyt_pd.iloc[i*split_num:(i+1)*split_num].reset_index(drop=True)
    sampled_nyt_pd.to_feather(f"./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/{i}_NYT_filtered_data.feather")

## Check

In [1]:
import pyarrow.feather as feather
fileid_list = []
for i in range(10):
    nyt_file = f"./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/{i}_NYT_filtered_data.feather"
    nyt_data = feather.read_feather(nyt_file)
    fileid_list.extend(nyt_data["file_id"].tolist())
print(len(set(fileid_list)))

FileNotFoundError: [Errno 2] Failed to open local file './0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/0_NYT_filtered_data.feather'. Detail: [errno 2] No such file or directory

In [3]:
nyt_data.head(5)

,file_id,pub,title,body_text
0,1144727,19991012,Fugitive Is Arrested Near Rome,"Giancarlo Parretti, a flamboyant former Hollyw..."
1,0137146,19880418,Dukakis's Record: A Success Story,Assessing the qualification of individuals to ...
2,0823851,19960119,Woe to Those Displaced by China Dam Project,"To the Editor:\nRe ""Cracks Show Early in China..."
3,0201339,19881125,Fugitive Greek Banker Is Arrested by the F.B.I.,"A fugitive Greek banker, indicted in a $300 mi..."
4,0404828,19901129,"In Boston, A Slaying Reawakens Gang Fears","They called themselves the Pistons, hoping a n..."


# Step3: Save docid2timestamp dict

**"./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/0_docid2timestamp.pkl"**

In [1]:
import pickle
import pyarrow.feather as feather

In [ ]:
docid2timestamp = dict()
for i in range(10):
    nyt_file = f"./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/{i}_NYT_filtered_data.feather"
    nyt_data = feather.read_feather(nyt_file)
    nyt_dict = nyt_data.set_index('file_id').to_dict()['pub']
    docid2timestamp.update(nyt_dict)
print(len(docid2timestamp))

In [3]:
with open("./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/0_docid2timestamp.pickle", 'wb') as f:
    pickle.dump(docid2timestamp, f)

In [8]:
import pickle
docid2timestamp = pickle.load(open("./0_Corpus/0_NYT_Data_Extraction/1_NYT_FilteredData_Files/0_docid2timestamp.pickle", "rb" ))
print(len(docid2timestamp))
print(docid2timestamp["0404828"])

1652347
19901129
